In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


import pandas as pd
app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [59]:
#drop the non-beneficial ID columns
app_df = app_df.drop(["EIN"], axis=1)

In [61]:
app_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [62]:
# Look at NAME value counts to identify and replace with "Other"
name_counts = app_df['NAME'].value_counts()

#Counts greater than 5
name_counts[name_counts>5]

NAME
PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: count, Length: 354, dtype: int64

In [64]:
#Determine which values to replace 
names_replace = list(name_counts[name_counts <= 5].index)

#Replace
for app in names_replace:
    app_df['NAME'] = app_df['NAME'].replace(app,"Other")

app_df['NAME'].value_counts()

NAME
Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: count, Length: 355, dtype: int64

In [69]:
app_counts = app_df['APPLICATION_TYPE'].value_counts()
app_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [70]:
app_replace = list(app_counts[app_counts <500].index)

for app in app_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

app_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [71]:
class_counts = app_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [72]:
class_replace = list(class_counts[class_counts < 1000].index)

for cls in class_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")

app_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [73]:
#Generate categorical variable lists
app_cat = app_df.dtypes[app_df.dtypes == "object"].index.tolist()

In [74]:
app_dummies_df = pd.get_dummies(app_df)

In [75]:
#Spliit data into features and target arrays
X= app_dummies_df.drop(["IS_SUCCESSFUL"], axis= 'columns').values
y = app_dummies_df["IS_SUCCESSFUL"].values


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [76]:
#create standardscaler instances
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

#Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [77]:
#deep neural net
num_features = len(X_train[0])
hn1 = 100
hn2 = 30
hn3 = 10

nn = tf.keras.models.Sequential()


nn.add(
    tf.keras.layers.Dense(units=hn1, input_dim= num_features, activation="relu")
)

nn.add(tf.keras.layers.Dense(units=hn2,activation="sigmoid"))

nn.add(tf.keras.layers.Dense(units=hn3, activation ="sigmoid"))

nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

nn.summary()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 100)            │        39,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 30)             │         3,030 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,251 (168.95 KB)

 Trainable params: 43,251 (168.95 KB)

 Non-trainable params: 0 (0.00 B)

In [78]:
#compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [79]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - accuracy: 0.6778 - loss: 0.5985
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 770us/step - accuracy: 0.7962 - loss: 0.4533
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - accuracy: 0.7937 - loss: 0.4433
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 773us/step - accuracy: 0.7959 - loss: 0.4353
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 768us/step - accuracy: 0.8016 - loss: 0.4232
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.7969 - loss: 0.4291
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step - accuracy: 0.7940 - loss: 0.4272
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 753us/step - accuracy: 0.7993 - loss: 0.4219
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - accuracy: 0.7956 - loss: 0.4239
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.7947 - loss: 0.4237
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step - accuracy: 0.8018 - loss: 0.4232
Epoch 12/100
804/80

In [80]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 791us/step - accuracy: 0.7883 - loss: 0.4494
Loss: 0.4494311511516571, Accuracy: 0.7883381843566895


In [81]:
nn.save("AlphabetSoupCharity_Optimization.h5")